In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
from gnomad.utils.plotting import *
from typing import Set, Tuple

tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
plot_dir = "/home/ubuntu/data/tmp"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.5
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.41-b8144dba46e6
LOGGING: writing to /home/ubuntu/data/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20201109-1632-0.2.41-b8144dba46e6.log


Loading BokehJS ...

In [2]:
ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_table_for_RF_by_variant_type.ht')

In [3]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        n_alt_alleles: int32, 
        QD: float64, 
        MQRankSum: float64, 
        SOR: float64, 
        ReadPosRankSum: float64, 
        FS: float64, 
        DP: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'QD': float64 
    'MQRankSum': float64 
    'SOR': float64 
    'ReadPosRankSum': float64 
    'FS': float64 
    'DP': int32 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_singleton': bool 
    'fail_hard_filters': bool 
    'ac_raw':

In [17]:
fp_expr = ht.fail_hard_filters
tp_expr = ht.omni | ht.mills | ht.kgp_phase1_hc | ht.hapmap

In [5]:
fp_expr.summarize()

Non-missing,13649567 (99.98%)
Missing,2242 (0.02%)
Counts,"{False: 13036114, True: 613453}"


In [18]:
tp_expr.summarize()

Non-missing,666248 (4.88%)
Missing,12985561 (95.12%)
Counts,{True: 666248}


In [7]:
truth_data_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/truthset_table.ht')

In [9]:
truth_data_ht.summarize()

Non-missing,30744091 (100.00%)
Missing,0
Contig Counts,"{'chr11': 1466650, 'chr5': 1981829, 'chr22': 364386, 'chr7_KI270803v1_alt': 2026, 'chr17_KI270862v1_alt': 8, 'chr8': 1708744, 'chr19': 554142, 'chrY': 2505, 'chr1': 2333219, 'chr15': 869740, 'chr12': 1424363, 'chr22_KI270928v1_alt': 1, 'chr18': 865799, 'chr8_KI270811v1_alt': 1, 'chr20': 709286, 'chr1_KI270706v1_random': 54, 'chr15_KI270850v1_alt': 1, 'chr8_KI270821v1_alt': 749, 'chr2': 2584084, 'chr17_KI270857v1_alt': 57, 'chr13': 1095602, 'chr14_KI270726v1_random': 1, 'chr14_KI270846v1_alt': 52, 'chr2_GL383522v1_alt': 1, 'chr7': 1698313, 'chr22_KI270875v1_alt': 1, 'chr14': 967788, 'chr1_KI270765v1_alt': 1, 'chr3': 2171623, 'chr17_KI270909v1_alt': 13, 'chr22_KI270879v1_alt': 264, 'chr17': 767196, 'chr14_GL000009v2_random': 86, 'chr1_KI270766v1_alt': 12, 'chr4': 2141421, 'chr2_KI270894v1_alt': 69, 'chr6': 1906190, 'chr9': 1277809, 'chr19_KI270938v1_alt': 90, 'chrX': 1068680, 'chr6_KI270801v1_alt': 1, 'chr4_GL000008v2_random': 75, 'chr10': 1471126, 'chr21': 401600, 'chr16': 908411, 'chrUn_KI270742v1': 22}"
Non-missing,30744091 (100.00%)
Missing,0
Min Size,1
Max Size,15
Mean Size,1.99
Non-missing,61167575 (100.00%)
Missing,0
Min Size,1


In [5]:
test_intervals="chr20"
if test_intervals:

        if isinstance(test_intervals, str):
            test_intervals = [test_intervals]
        test_intervals = [
            hl.parse_locus_interval(x, reference_genome="GRCh38")
            for x in test_intervals
        ]
        print(hl.eval(test_intervals))

[Interval(start=Locus(contig=chr20, position=1, reference_genome=GRCh38), end=Locus(contig=chr20, position=64444167, reference_genome=GRCh38), includes_start=True, includes_end=True)]


In [10]:
test_expr = hl.literal(test_intervals).any(
        lambda interval: interval.contains(ht.locus))
print("test_expr")


test_expr


In [14]:
ht.locus.summarize()

Non-missing,13651809 (100.00%)
Missing,0
Contig Counts,"{'chr5': 1454781, 'chr8': 1384, 'chr19': 1485, 'chr1': 2997581, 'chr20': 785942, 'chr2': 2274809, 'chr7': 1562341, 'chr3': 1793400, 'chr4': 1303848, 'chr6': 1476238}"


In [13]:
hl.literal(test_expr)

ExpressionException: 'literal' can only accept scalar or global expression arguments, found indices {'row'}

In [2]:
trio_stats_table = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_trios_stats.ht')

In [3]:
trio_stats_table.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'n_transmitted_raw': int64 
    'n_untransmitted_raw': int64 
    'n_transmitted_adj': int64 
    'n_untransmitted_adj': int64 
    'n_de_novos_raw': int64 
    'n_de_novos_adj': int64 
    'ac_parents_raw': int64 
    'an_parents_raw': int64 
    'ac_children_raw': int64 
    'an_children_raw': int64 
    'ac_parents_adj': int64 
    'an_parents_adj': int64 
    'ac_children_adj': int64 
    'an_children_adj': int64 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [5]:
trio_stats_table.n_transmitted_raw.summarize()

Non-missing,6462038 (100.00%)
Missing,0
Minimum,0
Maximum,9840
Mean,16.11
Std Dev,219.51


In [6]:
trio_stats_table.n_transmitted_adj.summarize()

Non-missing,6462038 (100.00%)
Missing,0
Minimum,0
Maximum,9820
Mean,11.45
Std Dev,177.65


In [7]:
trio_stats_table.ac_children_raw.summarize()

Non-missing,6462038 (100.00%)
Missing,0
Minimum,0
Maximum,19682
Mean,44.38
Std Dev,634.22


In [8]:
trio_stats_table.ac_children_adj.summarize()

Non-missing,6462038 (100.00%)
Missing,0
Minimum,0
Maximum,19672
Mean,25.26
Std Dev,474.08
